# Computing Closed-form Formulas

As explained in the README.md Polar can compute closed-form formulas for moments of program variables.
The formulas are parameterized by the number of loop iterations `n` and give you the moments after `n` many loop iterations.

## Command Line Interface

Let's compute the expected values and variances (second central moments) for the program variables `x` and `y` for the probabilistic loop in `loops/loop.prob` over the command line.

In [3]:
!python ../polar.py loops/loop.prob --goals "E(x)" "E(y)" "c2(x)" "c2(y)"


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



-------------------
- Analysis Result -
-------------------

E(x) = 1; n**3/256 + 133*n**2/256 + 205*n/128 + 1
Solution is exact

E(y) = 0; -n/8
Solution is exact

c2(x) = 0; 419*n**5/65536 + 1381*n**4/3072 + 313165*n**3/196608 + 291097*n**2/98304 + 131467*n/24576
Solution is exact

c2(y) = 0; 87*n/64
Solution is exact

Elapsed time: 1.4715957641601562 s


For every moment we passed in the `goals` parameter Polar outputs some initial values and a formula parameterized the number of loop iterations `n`.

## In Python

We can perform the same computation in our own Python files by importing Polar.

First we have to load the program:

In [7]:
from inputparser import Parser

program = Parser().parse_file("loops/loop.prob")
print(program)

_t2 = 1
_t3 = 0
x = _t2
y = _t3
while true:
    c1 = Bernoulli(1/2)
    c2 = Bernoulli(1/2)
    if c1 + c2 < 2:
        y = 1 + y {1/2} -2 + y {1/3} y
        g = Normal(y, 1)
        x = x + g**2
end


Next we have to convert the program into its "normal form" such that Polar can analyze it further.

In [9]:
from program import normalize_program

program = normalize_program(program)
print(program)

types
    c1 : Finite(0, 1)
    c2 : Finite(0, 1)
    _r5 : Finite(0, -2, -1)
end
x = 1
y = 0
while true:
    c1 = Bernoulli(1/2)
    c2 = Bernoulli(1/2)
    _r5 = -2 + c1 + c2
    y = 1 + y {1/2} -2 + y {1/3} y  |  (_r5 == -2 ∨ _r5 == -1)  :  y
    _u4 = Normal(0, 1)  |  (_r5 == -2 ∨ _r5 == -1)  :  _u4
    g = _u4 + y  |  (_r5 == -2 ∨ _r5 == -1)  :  g
    x = x + g**2  |  (_r5 == -2 ∨ _r5 == -1)  :  x
end


Now, we can use some functionality provided by Polar to construct a system of recurrences modelling the expected values of monomials in program variables.
We then solve the recurrences to obtain the closed-form formulas.

In [10]:
from recurrences import RecBuilder
from recurrences.solver import RecurrenceSolver

rec_builder = RecBuilder(program)
monomials = ["x", "y", "x**2", "y**2"]
closed_forms = {}
for monomial in monomials:
    # Construct the recurrences describing E(monomial) -> expected value of monomial
    recurrences = rec_builder.get_recurrences(monomial)
    # solve and save the closed-forms
    closed_forms[monomial] = RecurrenceSolver(recurrences).get(monomial)
print(closed_forms)

{'x': Piecewise((1, n <= 0), (n**3/256 + 133*n**2/256 + 205*n/128 + 1, True)), 'y': Piecewise((0, n <= 0), (-n/8, True)), 'x**2': Piecewise((1, n <= 0), (n**6/65536 + 685*n**5/65536 + 143911*n**4/196608 + 641881*n**3/196608 + 645391*n**2/98304 + 210187*n/24576 + 1, True)), 'y**2': Piecewise((0, n <= 0), (n*(n + 87)/64, True))}


The closed-forms are sympy expressions that contain the initial values and the closed-form formulas. It is possible that more than one initial values are listed before the general formula. The whole expressions describe the expected values of the monomials parameterized by the number of loop iterations `n`. The dictionary contains the first and second raw moments of `x` and `y`. If we want the variances we could compute it from the first and second moments.